In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path':  ChromeDriverManager().install()} #'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\zoomd\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [ ]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8812/5-hidden-gems-are-riding-aboard-nasas-perseverance-rover/" target="_self">5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover</a></div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

### Featured Images


In [8]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [10]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17936_hires.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_hires.jpg'

In [14]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [16]:
browser.quit()

### Mars Weather

In [18]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57556): Max retries exceeded with url: /session/803ec92352ade7de5c80313658cbae5f/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027E7CA33E88>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [ ]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [3]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [39]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
image_urls = []
titles = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
mars_hemispheres_soup = soup(html, 'html.parser')
product_list = mars_hemispheres_soup.find_all('div', class_='description')
base_url = 'https://astrogeology.usgs.gov'
# product_list

# loop through product_list
for product in product_list:
    link = product.a['href']
    prod_title = product.h3.text
    titles.append(prod_title)
    # goto link for summary page
    browser.visit(base_url + link)

    # Find and click the open full image button
    open_full_image_button = browser.find_by_id('wide-image-toggle')
    open_full_image_button.click()

    # Find the relative image url
    html = browser.html
    image_soup = soup(html, 'html.parser')
    image_container = image_soup.find_all('div', class_='downloads')
    
    for image_url in image_container:
        url_link = image_url.a['href']
        image_urls.append(url_link)
     
    
for x in image_urls:
    for y in titles:
        my_dicts = {  
                'image_url': x,
                'title': y
            }
        
    hemisphere_image_urls.append(my_dicts)


In [40]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'},
 {'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
# 5. Quit the browser
browser.quit()